This script converts DICOM image scans to resampled, segmented, masked 3D arrays and saves them for later feeding into the CNN

In [1]:
%matplotlib inline

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dicom
import os
import scipy.ndimage
import matplotlib.pyplot as plt

from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

# Some constants 
INPUT_FOLDER = '/Volumes/kaylab/Personal.Folders/BO/Python/DSB2017/sample_images/'
patients = os.listdir(INPUT_FOLDER)
patients.sort()
# for some odd reason there are duplicate files for each folder. Each duplicate starts with '._'
for i in range(len(patients)):
    if '._' in patients[i]:
        patients[i] = []
patients = list(filter(None,patients))

In [137]:
# Define all functions

# Load the scans in given folder path
def load_scan(path):
    # edited to skip all files starting with '._'
    # Also, file is missing 'DICM' marker. Use force=True to force reading
    slices = [dicom.read_file(path + '/' + s, force=True) for s in os.listdir(path) if '._' not in s]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    # Modification by Bo: Some rare slices are smaller than 512*512. Delete these slices
    # Make sure the slice thickness of the slice before the deletion is twice as thick
    rind = []
    for ii in range(len(slices)):
        if len(slices[ii].PixelData)==2*slices[ii].Rows**2:
            slices[ii].SliceThickness = slice_thickness
        else:
            rind.append(ii) # save index of slice to be removed
    
    if len(rind)==1:
        slices[rind[0]-1].SliceThickness = 2*slice_thickness
        del slices[rind[0]]
    elif len(rind)>1:
        for ii in sorted(rind,reverse=True): # if multiple bad slices delete in reverse
            slices[ii-1].SliceThickness = 2*slice_thickness
            del slices[ii]
    
    return slices

def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

# Resampling
def resample(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    spacing = map(float, ([scan[0].SliceThickness] + scan[0].PixelSpacing))
    spacing = np.array(list(spacing))

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor, mode='nearest')
    
    return image, new_spacing

# Lung segmentation
def largest_label_volume(im, bg=-1):
    vals, counts = np.unique(im, return_counts=True)
    # the background (air) label is -1
    counts = counts[vals != bg]
    vals = vals[vals != bg]

    if len(counts) > 0:
        return vals[np.argmax(counts)]
    else:
        return None

def segment_lung_mask(image, fill_lung_structures=True):
    
    # not actually binary, but 1 and 2. 
    # 0 is treated as background, which we do not want
    binary_image = np.array(image > -320, dtype=np.int8)+1
    labels = measure.label(binary_image)
    # measure.label assigns unique numerical values to unconnected components
    
    # Pick the pixel in the very corner to determine which label is air.
    #   Improvement: Pick multiple background labels from around the patient
    #   More resistant to "trays" on which the patient lays cutting the air 
    #   around the person in half
    imdim = labels.shape[1] # added by Bo
    background_label = labels[0,0,0]
    background_label2 = labels[0,imdim-1,0]# added by Bo
    background_label3 = labels[0,0,imdim-1]# added by Bo
    background_label4 = labels[0,imdim-1,imdim-1]# added by Bo
    
    #Fill the air around the person
    binary_image[background_label == labels] = 2
    binary_image[background_label2 == labels] = 2# added by Bo
    binary_image[background_label3 == labels] = 2# added by Bo
    binary_image[background_label4 == labels] = 2# added by Bo
    
    
    # Method of filling the lung structures (that is superior to something like 
    # morphological closing)
    if fill_lung_structures:
        # For every slice we determine the largest solid structure
        for i, axial_slice in enumerate(binary_image):
            axial_slice = axial_slice - 1
            labeling = measure.label(axial_slice)
            l_max = largest_label_volume(labeling, bg=-1)
            
            if l_max is not None: #This slice contains some lung
                binary_image[i][labeling != l_max] = 1

    
    binary_image -= 1 #Make the image actual binary
    binary_image = 1-binary_image # Invert it, lungs are now 1
    
    # Remove other air pockets insided body
    labels = measure.label(binary_image, background=0)
    l_max = largest_label_volume(labels, bg=-1)
    if l_max is not None: # There are air pockets
        binary_image[labels != l_max] = 0
 
    return binary_image

In [167]:
OUT_FOLDER = INPUT_FOLDER + '3Darrays'
if not os.path.exists(OUT_FOLDER):
    os.makedirs(OUT_FOLDER)
    
# Run the loop over all patients
for i in range(len(patients)):
    patientINFO = load_scan(INPUT_FOLDER + patients[i])
    print('Preprocessing '+patients[i]+' || '+str(len(patients)-i)+' remain')
    patient_pixels = get_pixels_hu(patientINFO)
    pix_resampled, spacing = resample(patient_pixels, patientINFO, [1,1,1])
    # create mask of filled lungs
    segmented_lungs_fill = segment_lung_mask(pix_resampled, True)
    # dilate mask to include any nodules that may appear on edge
    kernel = np.ones((5,5),np.uint8)
    dilated_mask = scipy.ndimage.morphology.binary_dilation(segmented_lungs_fill, iterations=6)
    # I find 6 iterations does the job
    masked_image = pix_resampled
    masked_image[dilated_mask==0] = 0 # mask the image by logical indexing
    outfile = os.path.join(OUT_FOLDER, '%s.npy' % patients[i])
    np.save(outfile, masked_image)
    

Preprocessing 0d941a3ad6c889ac451caf89c46cb92a || 3 remain
Preprocessing 0ddeb08e9c97227853422bd71a2a695e || 2 remain
Preprocessing 0de72529c30fe642bc60dcb75c87f6bd || 1 remain
